In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torchvision import transforms
import pandas as pd
import numpy as np

class HotelDataset(Dataset):
    def __init__(self, dataframe, numerical_features):
        self.dataframe = dataframe
        self.numerical_features = numerical_features
        self.tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path = self.dataframe['image'][idx]
        image = self.load_image(image_path)
        text_reviews = self.dataframe['text_reviews'][idx]
        text_description = self.dataframe['text_description'][idx]
        numerical_features = self.dataframe[self.numerical_features].iloc[idx].values.astype(np.float32)
        label = self.dataframe['price'][idx]
        encoded_text_reviews = self.tokenizer(text_reviews, padding='max_length', truncation=True, max_length=128,
                                              return_tensors='pt')
        encoded_text_description = self.tokenizer(text_description, padding='max_length', truncation=True,
                                                   max_length=128, return_tensors='pt')
        return {'image': image, 'text_reviews': encoded_text_reviews, 'text_description': encoded_text_description,
                'numerical_features': numerical_features, 'label': label}

    def load_image(self, image_path):
        ###############################################################################
        # Load image                                                                  #
        # and apply any necessary image preprocessing (e.g., resizing, normalization) #
        ###############################################################################
        image = None
        # Your image loading and preprocessing code here
        return image


class HotelPricePredictionModel(torch.nn.Module):
    def __init__(self, cnn_model, transformer1_model, transformer2_model, num_numerical_features, 
                 cnn_hidden_dim_size, distilbert_hidden_size, hidden_dim=128):
        super(HotelPricePredictionModel, self).__init__()
        self.cnn_model = cnn_model
        self.transformer1_model = transformer1_model
        self.transformer2_model = transformer2_model
        self.fc1 = torch.nn.Linear(2 * distilbert_hidden_size + cnn_hidden_dim_size + num_numerical_features, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, 1)

    def forward(self, image, text_reviews, text_description, numerical_features):
        # get the embeddings output by the 3 models
        cnn_embed = self.cnn_model(image)
        transformer1_embed = self.transformer1_model(**text_reviews)[0]
        transformer2_embed = self.transformer2_model(**text_description)[0]
        
        # reshape them to size (N, 1) for concatenating
        cnn_embed = cnn_embed.view(cnn_embed.size(0), -1)
        transformer1_embed = transformer1_embed.view(transformer1_embed.size(0), -1)
        transformer2_embed = transformer2_embed.view(transformer2_embed.size(0), -1)
        
        # concatenate embeddings with numerical features and pass through fully connected network
        concat_embed = torch.cat([cnn_embed, transformer1_embed, transformer2_embed, numerical_features], dim=1)
        
        x = torch.relu(self.fc1(concat_embed))
        x = self.fc2(x)
        return x.squeeze()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Define hyperparameters
learning_rate = 0.001
batch_size = 32
num_epochs = 10


cnn_path = ""
review_transformer_path = ""
description_transformer_path = ""
numerical_features = []
dataframe = pd.read_csv("https://raw.githubusercontent.com/john-zhang-uoft/hotel_price_prediction/main/data/final_data.csv")

cnn = torch.load(cnn_path)
review_transformer = AutoModelForSequenceClassification.from_pretrained(review_transformer_path)
description_transformer = AutoModelForSequenceClassification.from_pretrained(description_transformer_path)

cnn_embedding_layer_size = -1
transformers_embedding_layer_size = -1

########################################################################################
########################################################################################
# PUT CODE HERE THAT CHANGES CNN, AND THE TRANSFORMERS GETS THE EMBEDDING LAYERS FOR YOUR MODELS
########################################################################################
########################################################################################

dataset = HotelDataset(dataframe, numerical_features)

train_data, temp_data = train_test_split(dataset, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model = HotelPricePredictionModel(cnn, review_transformer, description_transformer, len(numerical_features), cnn_embedding_layer_size,
transformers_embedding_layer_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
        image = batch['image'].to(device)
        text_reviews = batch['text_reviews'].to(device)
        text_description = batch['text_description'].to(device)
        numerical_features = batch['numerical_features'].to(device)
        label = batch['label'].to(device)

        optimizer.zero_grad()

        output = model(image, text_reviews, text_description, numerical_features)

        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * image.size(0) # size of one batch

    epoch_loss = running_loss / len(train_data)
    train_losses.append(epoch_loss)

    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for batch in val_dataloader:
            image = batch['image'].to(device)
            text_reviews = batch['text_reviews'].to(device)
            text_description = batch['text_description'].to(device)
            numerical_features = batch['numerical_features'].to(device)
            label = batch['label'].to(device)

            output = model(image, text_reviews, text_description, numerical_features)
            val_loss = criterion(output, label)
            running_val_loss += val_loss.item() * image.size(0)

        val_epoch_loss = running_val_loss / len(val_data)
        val_losses.append(val_epoch_loss)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_epoch_loss:.4f}')

# Plot batch and validation loss


In [ ]:
# Plot batch and validation loss
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()
